# Libs

In [ ]:
from datetime import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
from collections import OrderedDict ,Counter
from functools import reduce 
from pyspark.sql.streaming import * 
from delta.tables import * 
from itertools import chain

# Input and Output

In [ ]:
# List the path of DBFS
dbutils.fs.ls("dbfs:/FileStore/landing_zone/Computer/Disconnected/Computer/")

In [ ]:
#Input
source_location = 'dbfs:/FileStore/landing_zone/Computer/Disconnected/Computer/'
#Output
target_location = 'dbfs:/FileStore/bronze/Computer/Disconnected/Computer/'
target_table = 'spark_catalog.bronze.computer_disconnected' 
checkpoint = 'dbfs:/FileStore/bronze/Computer/Disconnected/Computer_chk/' # Define the directory of checkpoint
schema = 'dbfs:/FileStore/bronze/Computer/Disconnected/Computer_schema/' # Define the Local schema
source = 'Computer Disconnected'

# Reading the streaming

In [ ]:
streamingDF = (spark.readStream.format('cloudFiles')\
    .option('cloudFiles.Format', 'parquet')\
    .option('cloudFiles.inferColumnTypes', 'true')\
    .option('cloudFiles.schemaLocation', schema)\
    .option('cloudFiles.schemaEvolutionMode', 'addNewColumns')\
    .load(source_location)\
     # Add the path   
     .withColumn('tracking_source',input_file_name())\
     # Add the source of information 
     .withColumn('source',lit(source)) \
     # Add the datetime from source data
     .withColumn("ingestionin", col("_metadata.file_modification_time")) \
     # Add a new column called "status" if necessary  
     .withColumn('status',lit(True)))

# Saving on a table target

In [ ]:
query = (streamingDF 
    .writeStream
    # .queryName("spark_catalog.bronze.celular_conectado")
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint)
    .option("path", target_location)
    .trigger(availableNow=True)
    .trigger(continuous='1 second') # Continuous process over a certain period
    .table(target_table)
    #.start()  #Finalizar o fluxo
    )
